In [5]:
import sounddevice as sd
import queue, json, time, statistics
import numpy as np
import noisereduce as nr
from vosk import Model, KaldiRecognizer

# ===== Params =====
SAMPLE_RATE = 16000
CHUNK_SIZE  = 2000      # 2000–4000; 2000 ≈ 125 ms blok
MODEL_PATH  = r"C:\Users\Admin\Desktop\vosk-model-small-en-us-0.15"

print("Loading Vosk model...")
model = Model(MODEL_PATH)
recognizer = KaldiRecognizer(model, SAMPLE_RATE)

q = queue.Queue()
latencies_partial = []
latencies_final   = []

def callback(indata, frames, time_info, status):
    if status:
        print(status, flush=True)
    # Blokun capture vaxtını daxil edirik
    q.put((bytes(indata), time.time()))

def main():
    print(" Listening... (Ctrl+C to stop)")
    with sd.RawInputStream(samplerate=SAMPLE_RATE, blocksize=CHUNK_SIZE,
                           dtype='int16', channels=1, callback=callback):
        while True:
            data, t_captured = q.get()

            # numpy
            audio = np.frombuffer(data, dtype=np.int16)

            # noise reduce (light)
            reduced = nr.reduce_noise(y=audio, sr=SAMPLE_RATE, prop_decrease=0.9, stationary=False, n_std_thresh_stationary=1.5)
            processed = reduced.astype(np.int16).tobytes()

            t_start = time.time()
            accepted = recognizer.AcceptWaveform(processed)
            t_decoded = time.time()

            if accepted:
                result = json.loads(recognizer.Result())
                text = result.get("text", "").strip()
                if text:
                    # Latency ölç: blok capture → final hazır
                    latencies_final.append(t_decoded - t_captured)
                    print(f" Final: {text}")
            else:
                partial = json.loads(recognizer.PartialResult())
                p = partial.get("partial", "").strip()
                if p:
                    # Partial latency: blok capture → partial hazır
                    latencies_partial.append(t_decoded - t_captured)
                    print(f"Partial: {p}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        # Session bitəndə latency statistikası
        def stat(arr):
            return {
                "count": len(arr),
                "avg_latency": round(1000 * (sum(arr)/len(arr)), 1) if arr else None,
                "median latency": round(1000 * statistics.median(arr), 1) if arr else None,
                "90th percentile latency": round(1000 * np.percentile(arr, 90), 1) if arr else None
            }
        print("\n--- Latency Report ---")
        print("Partial:", stat(latencies_partial))
        print("Final  :", stat(latencies_final))
        print("Finished!")



Loading Vosk model...
 Listening... (Ctrl+C to stop)
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
Partial: for example
 Final: for example
Partial: i
Partial: i
Partial: i am
Partial: i am
Partial: i am a
Partial: i am a
Partial: i am a
Partial: i am a
Partial: i am a program
Partial: i am a program
Partial: i am a program
Partial: i am a program
 Final: i am a program
Partial: machete
Partial: machete
Partial: the church in the
Partial: the church in the
Partial: the church in the
Partial: the church in the
Partial: the church in there is a
Partial: the church in there is a
Partial: the church in there is a
Partial: the church in there is a
Partial: the church in the desert a
Partial: the church in the desert a
Partial: the church in the desert a
Partial: the church in the desert a
Partial: the church in the desert a
 Final: the church in the 